## DCGAN で PyTorch実装を学ぶ
実装内容を復習する  
- Reference
  - https://github.com/eriklindernoren/PyTorch-GAN
  - https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/dcgan/dcgan.py

In [1]:
import os
import datetime
import numpy as np
import pandas as pd

In [2]:
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

In [4]:
import torch.nn as nn
#import torch.nn.functional as F
import torch

In [5]:
#torch.__version__

In [11]:
DATA_DIR_PATH = '/content/drive/My Drive/project/ML/pytorch-gan/data'

OUTPUT_DIR_PATH = os.path.join(DATA_DIR_PATH, '05_out')

In [12]:
# os.makedirs(os.path.join(OUTPUT_DIR_PATH, 'images'), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR_PATH, 'saved_models'), exist_ok=True)

In [33]:
cuda = True if torch.cuda.is_available() else False
print(cuda)

False


In [14]:
latent_dim = 100
img_size = 32
channels = 1

In [28]:
# Configure data loader
MNIST_DATA_DIR_PATH = os.path.join(DATA_DIR_PATH, 'mnist')

batch_size = 1

os.makedirs(MNIST_DATA_DIR_PATH, exist_ok=True)

dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        MNIST_DATA_DIR_PATH,
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ]),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [16]:
# def weights_init_normal(m):
#     classname = m.__class__.__name__
#     if classname.find("Conv") != -1:
#         torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
#     elif classname.find("BatchNorm2d") != -1:
#         torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
#         torch.nn.init.constant_(m.bias.data, 0.0)

In [21]:
class Generator(nn.Module):
    def __init__(self, img_size, latent_dim, channels):
        super(Generator, self).__init__()

        self.img_size = img_size
        self.latent_dim = latent_dim
        self.channels = channels

        self.init_size = self.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(self.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, self.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [17]:
init_size = img_size // 4
init_size

8

In [18]:
print(init_size ** 2)
print(128 * init_size ** 2)

64
8192


In [19]:
linear = nn.Linear(latent_dim, 128 * init_size ** 2)
linear

Linear(in_features=100, out_features=8192, bias=True)

In [20]:
l1 = nn.Sequential(nn.Linear(latent_dim, 128 * init_size ** 2))
l1

Sequential(
  (0): Linear(in_features=100, out_features=8192, bias=True)
)

In [22]:
nb2d = nn.BatchNorm2d(128)
nb2d

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [29]:
print(next(iter(dataloader)))

[tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]]), tensor([4])]


In [30]:
X, t = next(iter(dataloader))
print(X.shape)
print(t.shape)

torch.Size([1, 1, 32, 32])
torch.Size([1])


In [39]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

z = Tensor(np.random.normal(0, 1, (X.shape[0], latent_dim)))
z

tensor([[-1.8665,  0.2094, -0.1809,  0.6700,  0.5921, -0.4822,  0.6751,  1.9413,
         -0.3849, -1.4487, -0.2528,  0.9631,  1.3366, -0.4609, -0.6366, -0.3466,
          0.6329, -0.4273, -0.5893, -0.8734, -1.0509, -0.3843, -0.1019,  0.1408,
         -0.9550,  0.3480,  0.2214,  0.5603, -0.9483,  0.0657,  0.3038, -0.3585,
          1.1823, -2.2992, -0.9342, -0.4665, -1.0695, -2.0173,  0.4168,  1.1150,
         -0.3980,  2.0668,  0.2458,  1.0678, -0.4249,  0.6011,  1.0199, -0.3984,
         -2.5494, -1.9865,  1.3887, -0.8009, -0.0064, -0.5522, -2.1481, -0.4533,
          0.7738, -0.7430,  3.3721,  0.3039,  1.0832, -0.9682,  0.0186, -0.9717,
          1.0787,  0.6295,  0.0154, -0.3619, -0.1346, -1.1753,  0.7685,  1.8022,
          0.9276,  2.5506,  1.0535,  0.7698, -0.7943,  0.6484, -0.6728,  0.0619,
         -1.1646,  0.8131,  0.6837, -0.8095, -0.0853,  0.6897,  0.3214,  1.3038,
          0.6627, -0.2559,  0.0096,  0.0542,  1.2959,  0.2493, -0.8229,  0.6483,
         -0.7630,  1.5756, -

In [41]:
z.shape

torch.Size([1, 100])

In [38]:
np.random.normal(0, 1, (1, 10))

array([[ 0.72829906, -0.1271928 ,  1.51522606,  1.57340492,  1.80505037,
        -1.49303231, -0.76033891, -0.76808192, -0.31729294,  0.74171979]])

In [40]:
out = l1(z)
out

tensor([[ 0.1736,  0.7825, -0.3422,  ..., -0.6222,  0.2707,  1.1878]],
       grad_fn=<AddmmBackward>)

In [42]:
linear(z)

tensor([[-0.9133,  0.7018,  0.5221,  ...,  0.7486, -0.8089, -0.1470]],
       grad_fn=<AddmmBackward>)

In [43]:
#out = out.view(out.shape[0], 128, init_size, init_size)
out.shape

torch.Size([1, 8192])

In [47]:
out2 = out.view(out.shape[0], 128, init_size, init_size)
out2.shape

torch.Size([1, 128, 8, 8])

In [46]:
1*128*8*8

8192

In [48]:
out2

tensor([[[[ 0.1736,  0.7825, -0.3422,  ...,  0.4598, -0.5329, -1.1983],
          [ 0.3129, -0.1243,  0.4354,  ...,  0.4709,  0.5267,  0.4585],
          [ 0.3016, -0.1199, -0.2741,  ...,  0.2285, -0.9014,  0.3266],
          ...,
          [-0.2515,  1.2529,  0.2088,  ..., -0.4405, -0.5330, -0.1869],
          [-0.1216,  0.1714, -0.1190,  ...,  0.5239,  0.2195, -1.7700],
          [ 0.0690, -0.5419, -0.9009,  ...,  0.7239, -0.8466,  0.0203]],

         [[ 0.4586,  0.0900,  0.5676,  ...,  0.5456,  0.7457,  0.2615],
          [-0.4259,  0.8382,  0.3260,  ...,  0.5086,  0.6634, -0.2157],
          [ 0.2215,  0.5011, -1.2018,  ...,  1.0297,  0.6160,  0.4963],
          ...,
          [-0.4333,  0.5416, -0.0224,  ...,  0.3012, -0.2576, -0.4925],
          [-0.3838,  0.8016,  0.5112,  ..., -0.4741, -0.0864, -0.1006],
          [-0.1121,  0.2857, -0.8111,  ...,  0.4656, -0.4926, -0.4282]],

         [[ 0.3235, -0.4059,  0.6705,  ..., -0.2477, -0.5380, -0.0025],
          [ 0.2098,  0.4709, -

In [50]:
out3 = nb2d(out2)
out3

tensor([[[[ 0.4118,  1.3611, -0.3925,  ...,  0.8580, -0.6898, -1.7274],
          [ 0.6290, -0.0527,  0.8199,  ...,  0.8753,  0.9623,  0.8560],
          [ 0.6114, -0.0459, -0.2863,  ...,  0.4974, -1.2644,  0.6504],
          ...,
          [-0.2510,  2.0947,  0.4667,  ..., -0.5457, -0.6900, -0.1503],
          [-0.0485,  0.4083, -0.0444,  ...,  0.9580,  0.4833, -2.6188],
          [ 0.2487, -0.7038, -1.2637,  ...,  1.2699, -1.1789,  0.1728]],

         [[ 0.8006,  0.1112,  1.0043,  ...,  0.9632,  1.3374,  0.4320],
          [-0.8536,  1.5103,  0.5525,  ...,  0.8940,  1.1834, -0.4604],
          [ 0.3572,  0.8800, -2.3044,  ...,  1.8684,  1.0949,  0.8710],
          ...,
          [-0.8673,  0.9557, -0.0990,  ...,  0.5061, -0.5388, -0.9781],
          [-0.7748,  1.4420,  0.8990,  ..., -0.9437, -0.2186, -0.2452],
          [-0.2666,  0.4773, -1.5738,  ...,  0.8137, -0.9783, -0.8579]],

         [[ 0.4104, -0.9189,  1.0428,  ..., -0.6307, -1.1598, -0.1837],
          [ 0.2031,  0.6790, -

In [51]:
out3.shape

torch.Size([1, 128, 8, 8])

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_size, channels):
        super(Discriminator, self).__init__()

        self.img_size = img_size
        self.channels = channels

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [
                     nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace=True),
                     nn.Dropout2d(0.25)
            ]

            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))

            return block

        self.model = nn.Sequential(
            *discriminator_block(self.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = self.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity